In [6]:
import os
import io
import streamlit as st
from PIL import Image, ImageChops, ImageEnhance, ExifTags
import numpy as np
import cv2
import tempfile
from dotenv import load_dotenv
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

# Load environment variables
load_dotenv("config/.env")

# ------------------------------
# Text Model: DistilBERT (TensorFlow-only)
# ------------------------------
@st.cache_data(ttl=3600)
def load_distilbert_model(model_path="models/distilbert_fake_news"):
    try:
        tokenizer = DistilBertTokenizer.from_pretrained(model_path)
        model = TFDistilBertForSequenceClassification.from_pretrained(model_path, from_pt=False)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"]
        )
        return tokenizer, model
    except Exception as e:
        st.error(f"Error loading DistilBERT TensorFlow model: {e}")
        return None, None

@st.cache_data(ttl=1800)
def predict_distilbert(text, tokenizer, model):
    encodings = tokenizer([text], truncation=True, padding=True, max_length=64, return_tensors="tf")
    preds = model(encodings, training=False)
    probs = tf.nn.softmax(preds.logits, axis=1).numpy()[0]
    label = "FAKE" if np.argmax(probs) == 0 else "TRUE"
    confidence = float(max(probs) * 100)
    return label, confidence, probs

# ------------------------------
# Image Analysis: ELA + Metadata
# ------------------------------
def error_level_analysis(img: Image.Image, quality=90):
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".jpg")
    img.save(temp_file.name, 'JPEG', quality=quality)
    compressed = Image.open(temp_file.name)
    ela = ImageChops.difference(img, compressed)
    extrema = ela.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    scale = 255.0 / max_diff if max_diff != 0 else 1
    ela = ImageEnhance.Brightness(ela).enhance(scale)
    return ela

def get_image_metadata(img: Image.Image):
    metadata = {}
    try:
        info = img._getexif()
        if info:
            for tag, value in info.items():
                decoded = ExifTags.TAGS.get(tag, tag)
                metadata[decoded] = value
    except Exception:
        pass
    return metadata

# ------------------------------
# Video Analysis: Keyframes
# ------------------------------
def extract_keyframes(video_path, threshold=30):
    cap = cv2.VideoCapture(video_path)
    success, prev_frame = cap.read()
    keyframes = []

    while success:
        success, frame = cap.read()
        if not success:
            break
        diff = cv2.absdiff(cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY),
                           cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))
        non_zero_count = np.count_nonzero(diff)
        if non_zero_count > threshold:
            keyframes.append(frame)
        prev_frame = frame
    cap.release()
    return keyframes

# ------------------------------
# Streamlit App
# ------------------------------
st.set_page_config(page_title="AI Misinformation Assistant", page_icon="🛡️", layout="wide")
st.title("🛡️ AI-Powered Misinformation Detection & Literacy Assistant")

# Load model
tokenizer, model = load_distilbert_model()
MODEL_OK = tokenizer is not None and model is not None

st.sidebar.header("Model Status")
if MODEL_OK:
    st.sidebar.success("DistilBERT TensorFlow model loaded ✅")
else:
    st.sidebar.warning("DistilBERT model not found. Make sure 'models/distilbert_fake_news' exists.")

# Tabs
tab1, tab2, tab3 = st.tabs(["📝 Text", "🖼️ Image", "🎬 Video"])

# --- Text Tab ---
with tab1:
    st.subheader("Check a piece of text")
    user_text = st.text_area("Paste the message / headline here:", height=160)
    if st.button("Analyze Text"):
        if not user_text.strip():
            st.error("Please paste some text.")
        elif MODEL_OK:
            label, confidence, probs = predict_distilbert(user_text, tokenizer, model)
            st.write(f"**Prediction:** {label}")
            st.write(f"**Confidence:** {confidence:.1f}%")
            st.write(f"**Probabilities:** Fake={probs[0]*100:.1f}%, True={probs[1]*100:.1f}%")
        else:
            st.warning("Model not loaded.")

# --- Image Tab ---
with tab2:
    st.subheader("Check an image")
    uploaded_file = st.file_uploader("Upload an image (jpg/png)", type=["jpg", "jpeg", "png"])
    if uploaded_file:
        img = Image.open(io.BytesIO(uploaded_file.read())).convert("RGB")
        st.image(img, caption="Uploaded image", use_column_width=True)

        st.info("🔎 Running ELA and metadata checks...")
        ela_img = error_level_analysis(img)
        st.image(ela_img, caption="Error Level Analysis (ELA)", use_column_width=True)

        metadata = get_image_metadata(img)
        if metadata:
            st.write("Metadata found:")
            st.json(metadata)
        else:
            st.info("No metadata found.")

# --- Video Tab ---
with tab3:
    st.subheader("Check a video")
    uploaded_video = st.file_uploader("Upload a video (mp4)", type=["mp4"])
    if uploaded_video:
        temp_video = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
        temp_video.write(uploaded_video.read())
        st.video(temp_video.name)

        st.info("🔎 Extracting keyframes...")
        keyframes = extract_keyframes(temp_video.name)
        st.write(f"Number of keyframes detected: {len(keyframes)}")
        if keyframes:
            st.image([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in keyframes[:5]],
                     caption=[f"Keyframe {i+1}" for i in range(min(5, len(keyframes)))],
                     width=200)

st.caption("Educational starter. Always verify with trusted sources.")


2025-09-21 13:09:37.115 No runtime found, using MemoryCacheStorageManager
2025-09-21 13:09:37.119 No runtime found, using MemoryCacheStorageManager


DeltaGenerator()

In [3]:
!pip install python-dotenv

  Using cached python_dotenv-1.1.1-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.1-py3-none-any.whl (20 kB)
